In [2]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np


In [28]:
epsilons = [0, .05, .1, .15, .2, .25, .3]
pretrained_model = "models/best_model_FCN_final.pth"
use_cuda=False
# Set random seed for reproducibility
torch.manual_seed(42)

In [85]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os
print(os.getcwd())

/home/enzoc/Escritorio/lab4ML/Training-Inference-Attacks-Synthesis-Robustness-


In [88]:

# Define the epsilon values
epsilons = [0, 0.05, 0.1, 0.15, 0.2]

# Load your pretrained models
pretrained_cnn_model_path = "models/best_model_CNN_final.pth"
pretrained_fcn_model_path = "models/best_model_FCN_final.pth"


# Load your pretrained models
pretrained_cnn_model = torch.load(pretrained_cnn_model_path)
pretrained_fcn_model = torch.load(pretrained_fcn_model_path)

# Assuming you have a DataLoader for your dataset
# Modify the dataset loading and model usage based on your implementation
# For simplicity, I'll assume you're using ImageFolder
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Replace 'your_dataset_path' with the path to your dataset
dataset = ImageFolder(root='./data_split/test/', transform=data_transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Function to perform FGSM attack
def fgsm_attack(model, image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon * sign_data_grad
    
    # Clip the perturbed image to ensure it stays within the valid pixel range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    
    return perturbed_image

# Function to evaluate the model on the test dataset
def test_model(model, dataloader, epsilon):
    model.eval()
    correct = 0
    total = 0
    
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs.requires_grad = True
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and compute gradient
        model.zero_grad()
        loss.backward()
        
        # Collect data gradient
        data_grad = inputs.grad.data
        
        # Perform FGSM attack
        perturbed_inputs = fgsm_attack(model, inputs, epsilon, data_grad)
        
        # Forward pass on the perturbed image
        perturbed_outputs = model(perturbed_inputs)
        
        # Check the accuracy on the perturbed image
        _, perturbed_predicted = torch.max(perturbed_outputs.data, 1)
        total += labels.size(0)
        correct += (perturbed_predicted == labels).sum().item()
    
    accuracy = correct / total
    return accuracy

# Perform FGSM attack for both CNN and FCN
accuracies_cnn = []
accuracies_fcn = []

for epsilon in epsilons:
    acc_cnn = test_model(pretrained_cnn_model, dataloader, epsilon)
    acc_fcn = test_model(pretrained_fcn_model, dataloader, epsilon)
    
    accuracies_cnn.append(acc_cnn)
    accuracies_fcn.append(acc_fcn)

# Plot the results
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(epsilons, accuracies_cnn, marker='o')
plt.title('CNN Adversarial Attack')
plt.xlabel('Epsilon')
plt.ylabel('Accuracy')
plt.ylim(0, 1)

plt.subplot(1, 2, 2)
plt.plot(epsilons, accuracies_fcn, marker='o')
plt.title('FCN Adversarial Attack')
plt.xlabel('Epsilon')
plt.ylabel('Accuracy')
plt.ylim(0, 1)

plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'models/best_model_CNN_final.pth'